In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import holidays

In [ ]:
def build_features(df):
    daily_df = (df.groupby('datetime').agg(total_sales=('Sales', 'sum'), avg_discount=('Discount', 'mean')).reset_index())

    # Fill all days
    all_days_range = pd.date_range(start=daily_df['datetime'].min(), end=daily_df['datetime'].max()) 
    all_days = pd.DataFrame({'datetime': all_days_range})
    # Merge with your daily data
    daily_df = all_days.merge(daily_df, on='datetime', how='left')

    # Replace NaN (days with no sales) by 0
    daily_df['total_sales'] = daily_df['total_sales'].fillna(0)

    # year
    daily_df['year'] = daily_df['datetime'].dt.year

    # month
    daily_df['month'] = daily_df['datetime'].dt.month
    daily_df['month_sin'] = np.sin(2 * np.pi * daily_df['month'] / 12)
    daily_df['month_cos'] = np.cos(2 * np.pi * daily_df['month'] / 12)
    daily_df.drop('month', axis = 1, inplace = True)

    # week
    daily_df['week_of_month'] = daily_df['datetime'].apply(lambda d: (d.day - 1) // 7 + 1)
    daily_df['week_of_month_sin'] = np.sin(2 * np.pi * daily_df['week_of_month'] / 12)
    daily_df['week_of_month_cos'] = np.cos(2 * np.pi * daily_df['week_of_month'] / 12)
    daily_df.drop('week_of_month', axis = 1, inplace = True)

    # day of week
    daily_df['day_of_week'] = daily_df['datetime'].dt.dayofweek 
    daily_df['day_of_week_sin'] = np.sin(2 * np.pi * daily_df['day_of_week'] / 12)
    daily_df['day_of_week_cos'] = np.cos(2 * np.pi * daily_df['day_of_week'] / 12)
    daily_df.drop('day_of_week', axis = 1, inplace = True)

    # is discount
    daily_df['is_discount'] = daily_df['avg_discount'] > 0


    # holiday
    us_holidays  = holidays.CountryHoliday('US', years=range(2013, 2020))
    canada_holidays = holidays.CountryHoliday('CA', years=range(2013, 2020))
    holiday_dates =list(canada_holidays.keys())
    holiday_dates.append(list(us_holidays.keys()))
    daily_df['is_holiday'] = daily_df['datetime'].dt.date.isin(holiday_dates)

    # lag1
    daily_df['lag1'] = daily_df['total_sales'].shift(1)

    # lag7
    daily_df['lag7'] = daily_df['total_sales'].shift(7)

    # Fill NaN avg discounts
    daily_df['avg_discount'] = daily_df['avg_discount'].fillna(0)
    
    return daily_df

In [ ]:
train_data_path = '../data/train_data.csv'
raw_df = pd.read_csv(train_data_path)
raw_df['datetime'] = pd.to_datetime(raw_df['datetime'])

df = build_features(raw_df)

# Start at day 7 to avoid NaNs in lag7
df = df.iloc[7:]

In [ ]:
features = [f for f in df.columns if f != 'datetime']
data = df[features].values


# Normalize
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

In [ ]:

# Create sequences
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data)-seq_length):
        x = data[i:i+seq_length]
        y = data[i+seq_length,0]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

SEQ_LENGTH = 30  # for example, use past 12 months
X, y = create_sequences(data_scaled, SEQ_LENGTH)

In [ ]:
# Model
input_size = X.shape[2]  # number of features

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size=50, num_layers=2, output_size=1):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

model = LSTMModel(input_size=input_size)


In [ ]:
# Data loader
from torch.utils.data import Dataset, DataLoader
import torch

class SalesDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = SalesDataset(X, y)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)


In [ ]:
# Loss function 
import torch.nn as nn
import torch.optim as optim

input_size = X.shape[2]      # number of features per day
hidden_size = 64
num_layers = 3
output_size = 1

model = LSTMModel(input_size, hidden_size, num_layers, output_size)

criterion = nn.MSELoss()           # regression loss
optimizer = optim.Adam(model.parameters(), lr=0.0001)


In [ ]:
num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0

    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()           # reset gradients
        y_pred = model(X_batch)         # forward pass
        loss = criterion(y_pred, y_batch)
        loss.backward()                 # backprop
        optimizer.step()                # update weights

        epoch_loss += loss.item() * X_batch.size(0)

    epoch_loss /= len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")


In [ ]:
# Test

test_data_path = '../data/test_data.csv'
raw_df = pd.read_csv(test_data_path)
raw_df['datetime'] = pd.to_datetime(raw_df['datetime'])

test_df = build_features(raw_df)

# Start at day 7 to avoid NaNs in lag7
test_df = test_df.iloc[7:]
data_test = test_df[features].values


# Normalize
y_min = test_df['total_sales'].min()
y_max = test_df['total_sales'].max()
data_scaled = scaler.transform(data_test)
X_test, y_test = create_sequences(data_scaled, SEQ_LENGTH)

test_dataset = SalesDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)


predictions = []
true_values = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        output = model(X_batch)
        predictions.append(output.cpu())
        true_values.append(y_batch.cpu())

# Concatenate all batches
predictions = torch.cat(predictions, dim=0)
true_values = torch.cat(true_values, dim=0)
y_pred_original = predictions * (y_max - y_min) + y_min
y_true_original = true_values * (y_max - y_min) + y_min


In [ ]:
import matplotlib.pyplot as plt

plt.plot(y_true_original.numpy(), label='Actual')
plt.plot(y_pred_original.numpy(), label='Predicted')
plt.legend()
plt.show()


In [ ]:
test_df['total_sales'].mean()
